In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train= x_train0.copy()
x_test = x_test.copy()

In [ ]:
#%%script false
# found rate = 0.01

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

train_f1s=[]
test_f1s =[]
range= []

#rates =[0.99,0.9,0.5,0.25,0.2, 0.1,0.05, 0.01,0.0075, 0.005, 0.0025]
rates = np.arange(5,401,5)
for rate in rates:
    print("",rate,1/rate)
    undersample = RandomUnderSampler(sampling_strategy=1/rate,random_state=42)
    x_train, y_train = undersample.fit_resample(x_train0, y_train0)

    print(x_train.shape)
    #print(y_train.value_counts())
    
    modelClf = DecisionTreeClassifier(random_state=42)
    parameters={'criterion': 'entropy', 'max_depth': 12, 'min_samples_leaf': 10, 'min_samples_split': 26}
    modelClf.set_params(**parameters)

    modelClf.fit(x_train, y_train)
    predsTrain = modelClf.predict(x_train)
    predsTest = modelClf.predict(x_test)

    train_f1=fd.calculate_scores(y_train, predsTrain,'f1')
    print("f1 train {:.4f}".format(train_f1))
    test_f1=fd.calculate_scores(y_test, predsTest,'f1')
    print("f1 test  {:.4f}".format(test_f1))
    
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(rate)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

rate=140
undersample = RandomUnderSampler(sampling_strategy=1/rate,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)
print(x_train.shape)  

    
modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': 'entropy', 'max_depth': 12, 'min_samples_leaf': 10, 'min_samples_split': 26}
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)


In [ ]:
# threshold no influence
y_pred_prob = modelClf.predict_proba(x_test)[:,1]

range = []
f1s = []
rocs = []
for threshold  in [0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6]:
    print("--- threshold --", threshold)
    y_pred_adjusted = (y_pred_prob>threshold).astype(int)
    f1,mcc,roc=fd.print_scores(y_test, y_pred_adjusted,'all')
    range.append(threshold)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s,"f1 score",rocs,"rocs score")

In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
threshold=0.5
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    y_pred_prob = modelClf.predict_proba(dfTrx[predictors])[:,1]
    y_pred_adjusted = (y_pred_prob>threshold).astype(int)

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], y_pred_adjusted,'All', True)
    
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], y_pred_adjusted)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('06-sklearn.tree','DecisionTreeClassifier','4-After undersampling', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

